# 4.1 인스타그램 크롤링 

### 4.1.2 인스타그램 검색 결과 URL을 만들어서 접속하기

In [1]:
# 예제 4-1 인스타그램 검색결과 URL을 만드는 함수
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
    

In [2]:
# 예제 4-2 selenium으로 URL 접속하기 - 1
from selenium import webdriver

driver = webdriver.Chrome("c:/chromedriver.exe")   

#### 안내사항
로그인을 해야만 게시물을 볼 수 있도록 인스타그램 운영 정책이 변경되었습니다(2020.04.27)  
인스타그램 로그인을 하는 코드를 추가하였습니다.   
직접 브라우저에 계정 정보를 입력해도 되고, 코드를 이용해도 됩니다.   
다만,  본인의 계정/비번 정보가 외부에 노출되지 않도록 주의하시기 바랍니다. 

※ 주의) `인스타 로그인`을 위해 계정 `아이디` / `비번` 입력이 필요합니다. 

In [3]:
# 예제 4-2 selenium으로 URL 접속하기 - 2
# 인스타그램 로그인 부분 추가(2020.05.07)
import time

# 인스타그램 접속하기
driver.get('https://www.instargram.com')
time.sleep(2)

In [5]:
# 예제 4-2 selenium으로 URL 접속하기 - 3
# 인스타계정으로 로그인하기
email = ''   ### 계정 정보 수정 필요
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '' ### 비번 정보 수정 필요
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

위 코드 실행후 인스타그램 로그인이 완료되어야 합니다.  

추가로, 2단계 인증이 필요하다고 나올 경우 인증까지 완료해주세요. 

인스타계정이 아닌 페이스북계정으로 로그인이 필요하거나  코드에 아이디/비번 입력하는 것이 부담되는 분은

수기로 직접 입력해서 로그인까지 완료하시면 됩니다. 



In [6]:
# 예제 4-2 selenium으로 URL 접속하기 - 4
word = "기념일맛집"
url = insta_searching(word)
driver.get(url)

### 4.1.3 첫 번째 게시글 열기 

In [8]:
# 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

### 4.1.4 게시글 정보 가져오기 

Mac 에서 작성된 글의 경우 크롤링 데이터 엑셀파일로 저장시,  한글 자음/모음이 분리되는 현상이 확인되었습니다.   
저장하는 과정에서 unicodedata를 이용하여  'NFC' 방식으로(자음/모음 합쳐서 한글 처리하는 방식) 전환하는 코드 추가되었습니다.   
한글이 사용되는 본문 및 장소명 부분에 해당 코드 추가하였습니다. (수정일 2020.08.20)

In [9]:
# 예제 4-4 게시글 정보 가져오기
import re
from bs4 import BeautifulSoup
import unicodedata   # 추가, 2020.08.20

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content)   # 신규 추가. 2020.08.20
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)      # 신규 추가. 2020.08.20
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

['최고의 인기를 끌고 있는 세트메뉴💕주말에도 많이 방문해주셔서 감사합니다🙏멕시코타코 요리의 진가를최선을 다해 보여드리겠습니다💙탁트인 공간에서 힐링하고 가세요',
 '2020-10-26',
 '1,318',
 '',
 []]

### 4.1.5 다음 게시글 열기

In [10]:
# 예제 4-5 다음 게시글 열기
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

move_next(driver)

### 4.1.6 여러 게시글 정보 수집하기

In [11]:
# 예제 4-6 인스타그램 크롤링  1 - 인스타그램 로그인하기
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# ① 크롬 브라우저 열기
driver = webdriver.Chrome('c:/chromedriver.exe') 

# 추가) 인스타그램 접속후, 로그인 하기  - 
driver.get('https://www.instargram.com')
time.sleep(2)


######## 인스타 계정 로그인이 필요합니다 #########
email = '인스타계정입력해주세요'   ### 계정 정보 수정 필요
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '인스타비번입력해주세요' ### 비번 정보 수정 필요
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

IndexError: list index out of range

위 코드 실행후 인스타그램 로그인이 완료되어야 합니다.  

추가로, 2단계 인증이 필요하다고 나올 경우 인증까지 완료해주세요. 

인스타계정이 아닌 페이스북계정으로 로그인이 필요하거나  코드에 아이디/비번 입력하는 것이 부담되는 분은

수기로 직접 입력해서 로그인까지 완료하시면 됩니다. 



In [ ]:
# 예제 4-6 인스타그램 크롤링  2 - 크롤링하기
# ② 인스타그램 검색페이지 URL 만들기
word = "기념일맛집"    #검색어
url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(3)

# ④ 첫 번째 게시글 열기
select_first(driver)

# ⑤ 비어있는 변수(results)만들기
results = [ ]


# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 500      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    

print(results[:2])

In [ ]:
# 예제 4-7 크롤링 결과 저장하기

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','date','like','place','tags']
results_df.to_excel('./files/3_1_crawling_jejudoMatJip.xlsx', index=False)


In [ ]:
results_df

### 4.1.8 여러 엑셀 파일의 중복을 제거한 후 통합 저장


In [ ]:
# 예제 4-8 여러 개의 저장파일 통합하기
jeju_insta_df = pd.DataFrame( [ ] )

folder = './files/'
f_list = ['3_1_crawling_jejudoMatJip.xlsx', '3_1_crawling_jejudoGwanGwang.xlsx', '3_1_crawling_jejuMatJip.xlsx', '3_1_crawling_jejuYeoHang.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns =['content','date','like','place','tags']


In [ ]:
jeju_insta_df

In [ ]:
# 예제 4-9 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = ["content"] , inplace = True)
jeju_insta_df.to_excel('./files/3_1_crawling_raw.xlsx', index = False, encoding="utf-8")

In [ ]:
jeju_insta_df